In [17]:
import os
from helper_functions import tree
from pathlib import Path
import pandas as pd
from forest.jasmine.traj2stats import gps_stats_main, Hyperparameters
from forest.constants import Frequency
import sys
import glob
import yaml
# TODO isolate get_subdirectories() to liz_helper_functions
# from liz_helper_functions import get_subdirectories
from tqdm import tqdm 
from dataclasses import dataclass
from datetime import datetime
import json
import logging
import os
import pickle
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from pyproj import Transformer
import requests
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import transform

from forest.bonsai.simulate_gps_data import bounding_box
from forest.constants import Frequency, OSM_OVERPASS_URL, OSMTags
from forest.jasmine.data2mobmat import (gps_to_mobmat, infer_mobmat,
                                        great_circle_dist,
                                        pairwise_great_circle_dist)
from forest.jasmine.mobmat2traj import (imp_to_traj, impute_gps, locate_home,
                                        num_sig_places)
from forest.jasmine.sogp_gps import bv_select
from forest.poplar.legacy.common_funcs import (datetime2stamp, read_data,
                                               stamp2datetime,
                                               write_all_summaries)
from forest.utils import get_ids



# Get the absolute path to the directory this script lives in
SCRIPT_DIR = os.getcwd() # os.path.dirname(os.path.abspath(__file__)) for script

# Construct the path to the config file relative to the script
CONFIG_PATH = os.path.join(SCRIPT_DIR, "../../config/old_HOPE_config.yaml")
# # quick and dirty hardcoding for testing
# CONFIG_PATH = "/n/onnela_dp_l3/Lab/HOPE/beiwe/config/HOPE_config.yaml"
CONFIG_DIR = os.path.dirname(CONFIG_PATH)

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# data_dir = os.path.abspath(os.path.join(CONFIG_DIR, config["data_dir"]))
raw_data_dir = os.path.abspath(os.path.join(CONFIG_DIR, config["raw_data_dir"]))
raw_data_folder = os.path.abspath(os.path.join(raw_data_dir, config["raw_data_folder"]))
metadata_path = os.path.abspath(os.path.join(CONFIG_DIR, config["metadata_path"]))

data_dir = os.path.abspath(os.path.join(CONFIG_DIR, config["data_dir"]))



dest_dir = raw_data_folder

beiwe_id_df = pd.read_csv(metadata_path, sep = ",")

In [19]:
file_path = file_path_array[i]  # The file that caused the error
with open(file_path, 'r') as f:
    for _ in range(5):
        print(f.readline())

NameError: name 'file_path_array' is not defined

In [18]:
# jasmine job 5450095 debugging

FILE_PATH = "/n/onnela_dp_l3/Lab/HOPE/beiwe/data/raw/manual_HOPE_paper2_download/ige4zl6o/gps"

In [25]:
def gps_quality_check(study_folder: str, study_id: str) -> float:
    """The function checks the gps data quality.

    Args:
        study_folder (str): The path to the study folder.
        study_id (str): The id code of the study.
    Returns:
        a scalar between 0 and 1, bigger means better data quality
            (percentage of data which meet the criterion)
    """
    gps_path = f"{study_folder}/{study_id}/gps"
    if not os.path.exists(gps_path):
        quality_check = 0.
    else:
        file_list = os.listdir(gps_path)
        for i, _ in enumerate(file_list):
            if file_list[i][0] == ".":
                file_list[i] = file_list[i][2:]
        file_path = [
            f"{gps_path }/{file_list[j]}"
            for j, _ in enumerate(file_list)
        ]
        file_path_array = np.sort(np.array(file_path))
        # check if there are enough data for the following algorithm
        quality_yes = 0.
        for i, _ in enumerate(file_path_array):
            df = pd.read_csv(file_path_array[i])
            if df.shape[0] > 60:
                quality_yes = quality_yes + 1.
        quality_check = quality_yes / (len(file_path_array) + 0.0001)
    return quality_check

In [26]:
BUGGY_ID = "ige4zl6o"

In [27]:
test = gps_quality_check(study_folder = raw_data_folder, study_id = BUGGY_ID)

ParserError: Error tokenizing data. C error: Expected 6 fields in line 3, saw 7


In [12]:
test

0.0

In [13]:
raw_data_folder

'/n/onnela_dp_l3/Lab/HOPE/beiwe/data/raw/manual_HOPE_paper2_download'